In [19]:
from MoNet.flowfunc import FuncModel as fn
import MoNet.monet as mn
import importlib
importlib.reload(mn)
# importlib.reload(fn)
import torch
import torch.nn as nn

monet 支持的接口：

- `Fn(args=[],call:str | Callable ='SEQ')`
- `X = mn.Fn()`
- `Adup(module,i=0,o=0,net='',mn_dict={},in_dim=-1)`
  - mn.Fn()
- `Layer(i=0,o_lists=[10,[32,32],1],net_lists=['dp_0.5',["fc",'bn','act','dp_0.5'],"fc"],mn_dict=mn_dict,in_dim=-1)`
  - Layer(i=0, o_lists=1, net_lists="fc_1") # 输入维度，输出维度，网络名称模式
  - Layer("fc_1", 1) # 省略输入维度的自适应模式
- `Fn().p()`

monet 支持的计算：

- \+
- \&
- \*
- \*\*

In [20]:
x=torch.randn(1,10)
# F = mn.Fn()*mn.X
# print(F);print(F(x));print(F)# 空的网络会直接返回输入的x
# F = mn.Layer(0,1,'fc')
# print(F);print(F(x));print(F) # 设置为0时会自动调整输入输出维度
F = mn.Layer(0,1,['fc','act']) # 输入 输出 网络
print(F);print(F(x));print(F)

[1, 1] ['fc', 'act']
SEQ>Fn(
  (0:fc): @Adup:Linear(in_features=0, out_features=1, bias=True) *id:1798851407456
  (1:act): @Adup:GELU(approximate='none') *id:1798851409040
)
[Warning] Input dim is not match, set to 10
[1] ['fc']
tensor([[-0.1489]], grad_fn=<GeluBackward0>)
SEQ>Fn(
  (0:fc): @Adup:Linear(in_features=10, out_features=1, bias=True) *id:1798851407456
  (1:act): @Adup:GELU(approximate='none') *id:1798851409040
)


In [22]:
# # 这四种等价，都是一个对一个
# print(mn.Layer('fc',10))
# print(mn.Layer('fc',[10]))
# print(mn.Layer(['fc'],10))
# print(mn.Layer(['fc'],[10]))

# # 第一维度，一一对应，缺少部分重复最后一个到一样长
# print(mn.Layer('fc',[10,20]))
# print(mn.Layer(['fc'],[10,20]))
# print(mn.Layer(['fc','act'],[10,20]))
# print(mn.Layer(['fc','act'],10))
# print(mn.Layer(['fc','act'],[10]))

# 在第二维度，网络代表当成一个整体，而输出代表重复
# print(mn.Layer(['fc','act'],[[10,20],30]))
# print(mn.Layer([['fc','bn'],'act'],[10,20]))
print(mn.Layer([['fc','bn'],'act'],[[10,20],30]))

# 再复杂的网络采用*或者+连接

[[10, 20], 30] [['fc', 'bn'], 'act']
[10, 20] [['fc', 'bn'], ['fc', 'bn']]
[10, 10] ['fc', 'bn']
[20, 20] ['fc', 'bn']
SEQ>Fn(
  (0:mix): SEQ>Fn(
    (cell-0): SEQ>Fn(
      (0:fc): @Adup:Linear(in_features=0, out_features=10, bias=True) *id:1798851409184
      (1:bn): @Adup:BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) *id:1798851409328
    )
    (cell-1): SEQ>Fn(
      (0:fc): @Adup:Linear(in_features=10, out_features=20, bias=True) *id:1798851408800
      (1:bn): @Adup:BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) *id:1798851410864
    )
  )
  (1:act): @Adup:GELU(approximate='none') *id:1798851409424
)


In [23]:
F=mn.Layer(0,[10,[0,10],1],['dp_0.5',['fc','act'],'fc'])
F1=F*(fn(max)+min)
F1

[10, [0, 10], 1] ['dp_0.5', ['fc', 'act'], 'fc']
[0, 10] [['fc', 'act'], ['fc', 'act']]
[0, 0] ['fc', 'act']
[10, 10] ['fc', 'act']


SEQ>Fn(
  (0): @Adup:Dropout(p=0.5, inplace=False) *id:1798851410864
  (1): SEQ>Fn(
    (cell-0): SEQ>Fn(
      (0:fc): @Adup:Linear(in_features=10, out_features=0, bias=True) *id:1798851405392
      (1:act): @Adup:GELU(approximate='none') *id:1798851410624
    )
    (cell-1): SEQ>Fn(
      (0:fc): @Adup:Linear(in_features=0, out_features=10, bias=True) *id:1798851409712
      (1:act): @Adup:GELU(approximate='none') *id:1798851407024
    )
  )
  (2): @Adup:Linear(in_features=10, out_features=1, bias=True) *id:1798851410384
  (3): LIC>Fn(
    (0): @dup:max *id:1798851407120
    (1): @dup:min *id:1798851406976
  )
)

In [24]:
F.p(torch.randn(2,10))

@ SEQ>Fn( >>
    tensor([[ 1.1471, -2.2010, -0.5649,  0.6266,  1.3925,  1.4525,  0.0057, -1.0800,
         -0.2058,  0.2350],
        [-0.7154,  0.0075,  0.2986,  0.8201,  0.3921,  0.0758, -0.1757, -0.0256,
          0.8291,  0.2075]])
[Warning] Input dim is not match, set to 10
[10] ['dp_0.5']
@ @Adup:Dropout(p=0.5, inplace=False) *id:1798851410864 >>
    tensor([[ 1.1471, -2.2010, -0.5649,  0.6266,  1.3925,  1.4525,  0.0057, -1.0800,
         -0.2058,  0.2350],
        [-0.7154,  0.0075,  0.2986,  0.8201,  0.3921,  0.0758, -0.1757, -0.0256,
          0.8291,  0.2075]])
  Net : Dropout(p=0.5, inplace=False)
  $ (tensor([[ 1.1471, -2.2010, -0.5649,  0.6266,  1.3925,  1.4525,  0.0057, -1.0800,
         -0.2058,  0.2350],
        [-0.7154,  0.0075,  0.2986,  0.8201,  0.3921,  0.0758, -0.1757, -0.0256,
          0.8291,  0.2075]]),) >>
  == tensor([[ 2.2941, -4.4021, -0.0000,  1.2533,  0.0000,  2.9049,  0.0000, -0.0000,
         -0.4117,  0.4699],
        [-0.0000,  0.0000,  0.0000,  0.00

tensor([[-0.1061],
        [-0.1061]], grad_fn=<AddmmBackward0>)